# Example: Bag of Words, TF-IDF and PMI
In this example, we'll play around with simple collections of text data and explore how to create basic text embeddings using the Bag of Words model, Term Frequency-Inverse Document Frequency (TF-IDF), and Pointwise Mutual Information (PMI).

> __Learning Objectives:__
> 
> By the end of this example, you should be able to:
>
> Three learning objectives go here.

Let's get started!
___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants.

> The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [ ]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


## Summary
One concise, direct summary sentence goes here.

> __Key Takeaways:__
> 
> Three key takeaways go here.

One concise, direct concluding sentence goes here.
___